In [121]:
import numpy as np

from gensim.models import KeyedVectors
from gensim.models import Word2Vec

from gensim.utils import simple_preprocess
from gensim.utils import simple_tokenize

In [73]:
# command line to convert glove file to word2vec file.
# python -m gensim.scripts.glove2word2vec --input <glove_file> --output <w2v_file>

def load_wv(fn='../data/word2vec/glove.twitter.27B/word2vec.50d.txt'):
    wv = KeyedVectors.load_word2vec_format(fn, binary=False)
    return wv

In [74]:
def preprocess(sentence):
    simple_tokenize(sentence)
    return sentence

In [75]:
def load_corpus(fn='../data/data.csv'):
    sentences = []
    with open(fn, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            sentence = line.split('\t')[0].strip()
#             sentence = sentence.split(' ')
            sentences.append(preprocess(sentence))
    return sentences

In [128]:
def build_model(sentences, dim, pretrain_fn='../data/word2vec/glove.twitter.27B/word2vec.50d.txt'):
    model = Word2Vec(size=dim, min_count=5)
    model.build_vocab(sentences)
    total_examples = model.corpus_count
    
    print('load pre-trained vectors...')
    glove_wv = load_wv(pretrain_fn)
    
    print('intersect glove vectors')
    model.build_vocab([list(glove_wv.vocab.keys())], update=True)
    model.intersect_word2vec_format(pretrain_fn, binary=False, lockf=1.0)
    
    print('train model...')
    model.train(sentences, total_examples=total_examples, epochs=model.iter)
    return model

In [77]:
def save_model_to(model, fn='../data/word2vec/retrained_word2vec/reddit_word2vec'):
    print('saving word2vec...')
    model.save(fn)

In [129]:
def load_model_with(fn='../data/word2vec/retrained_word2vec/reddit_word2vec'):
    print('loading word2vec...')
    model = Word2Vec.load(fn)
    return model

In [82]:
sentences = load_corpus()
model = build_model(sentences, 50)
save_model_to(model)

load pre-trained vectors...
train model...


/Users/zhengcao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  del sys.path[0]


In [84]:
model.similar_by_word('students', topn=10)

/Users/zhengcao/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('teachers', 0.9260638356208801),
 ('schools', 0.8740049004554749),
 ('student', 0.8569083213806152),
 ('college', 0.8254045248031616),
 ('colleges', 0.8141670227050781),
 ('professors', 0.8069064617156982),
 ('classes', 0.7910361289978027),
 ('graduate', 0.7838039994239807),
 ('education', 0.7693856358528137),
 ('seniors', 0.7657411098480225)]

In [86]:
new_model = load_model_with()